In [1]:
import pandas as pd

In [3]:
import openpyxl
import numpy as np
import shapely
from shapely.geometry import Point, Polygon, box, GeometryCollection, MultiPolygon
import pyproj
import pickle


In [127]:
xl=pd.read_excel('E:/Users/sentinel_industry/Global-Coal-Plant-Tracker-Jan-2022.xlsx', sheet_name=1)

In [24]:
len(pd.unique(xl.loc[xl.Status=='operating'].loc[xl.Accuracy=='exact'].Latitude))

2415

In [128]:
df=xl.loc[xl.Status=='operating'].loc[xl.Accuracy=='exact']
df2 = df.drop_duplicates(subset=["Latitude", "Longitude"], keep='first')

In [129]:
df2=df2.set_index('Tracker ID')

In [130]:
df2.Latitude[0]

-22.5195

In [136]:
for i in range(len(df2)):
    if i%100==1:print(i/len(df2))
    make_aoi_shape_from_latlons(df2.Latitude[i],df2.Longitude[i]+0.1, df2.index[i], 'E:/Users/sentinel_industry/polys/coal/no_ind/')

0.00041356492969396195
0.041770057899090156
0.08312655086848635
0.12448304383788254
0.16583953680727875
0.20719602977667495
0.24855252274607112
0.28990901571546734
0.33126550868486354
0.37262200165425974
0.41397849462365593
0.45533498759305213
0.49669148056244833
0.5380479735318445
0.5794044665012407
0.6207609594706369
0.6621174524400331
0.7034739454094293
0.7448304383788255
0.7861869313482217
0.8275434243176178
0.8688999172870141
0.9102564102564102
0.9516129032258065
0.9929693961952026


In [ ]:
svpath='E:/Users/sentinel_industry/polys/coal/'

In [19]:
def make_aoi_shape_from_latlons(la, lo, tracker_id, savepath):
        la_dist = 1/111.32 * 1.2  
        lo_dist = 1/np.cos(la*np.pi/180) * la_dist
        coords=((lo-lo_dist, la-la_dist),(lo-lo_dist, la+la_dist),(lo+lo_dist, la+la_dist),(lo+lo_dist, la-la_dist))
        shape= Polygon(coords)
        with open(savepath+tracker_id, "wb") as poly_file:
            pickle.dump(shape, poly_file, pickle.HIGHEST_PROTOCOL)

In [7]:
la=xl.Latitude[0]
lo=xl.Longitude[0]

In [21]:

make_aoi_shape_from_latlons(xl.Latitude[0],xl.Longitude[0], xl['Tracker ID'][0], 'E:/Users/sentinel_industry/polys/coal/')

## Steel Plants

In [70]:
xl=pd.read_excel('E:/Users/sentinel_industry/Global-Steel-Plant-Tracker-March-2022_corrected.xlsx', sheet_name=1)

In [67]:
float(xl.Coordinates[0].split(', ')[0])

36.753961

In [62]:
df=xl.loc[xl.Status=='operating'].loc[xl['Coordinate accuracy']=='exact']
df3 = df.drop_duplicates(subset=["Coordinates"], keep='first')

In [134]:
for k,i in enumerate(df3.index):
    if k%100==1:print(k/len(df3))
    try: make_aoi_shape_from_latlons(float(df3.Coordinates[i].split(', ')[0]), float(df3.Coordinates[i].split(', ')[1])+0.1 , df3['Plant ID'][i], 'E:/Users/sentinel_industry/polys/steel/no_ind/')
    except ValueError: continue

0.0012091898428053204
0.12212817412333736
0.2430471584038694
0.3639661426844015
0.4848851269649335
0.6058041112454655
0.7267230955259976
0.8476420798065296
0.9685610640870617


In [74]:
xl_a=pd.read_excel('E:/Users/sentinel_industry/SFI-Global-Steel-Database-July-2021.xlsx', sheet_name=1)

In [76]:
xl_a.keys()

Index(['uid', 'city', 'state', 'country', 'iso3', 'country_code', 'region',
       'sub_region', 'latitude', 'longitude', 'accuracy', 'status',
       'plant_type', 'primary_production_type', 'primary_product', 'capacity',
       'capacity_source', 'year', 'owner_permid', 'owner_name', 'owner_source',
       'parent_permid', 'parent_name', 'ownership_stake', 'parent_lei',
       'parent_holding_status', 'parent_ticker', 'parent_exchange',
       'parent_permid_2', 'parent_name_2', 'ownership_stake_2', 'parent_lei_2',
       'parent_holding_status_2', 'parent_ticker_2', 'parent_exchange_2'],
      dtype='object')

In [81]:
df=xl_a.loc[xl_a.status=='Operating'].loc[xl_a.accuracy=='Exact']
df4 = df.drop_duplicates(subset=["latitude", "longitude"], keep='first')

In [96]:
def extract_la(row):
    lalo=row.Coordinates.split(',')
    la=lalo[0]
    return float(la)
def extract_lo(row):
    lalo=row.Coordinates.split(',')
    lo=lalo[1]
    return float(lo)

In [95]:
df3['Latitude'] = df3.apply(extract_la, axis=1)
df3['Longitude'] = df3.apply(extract_lo, axis=1)

In [117]:
(np.round(df3['Latitude'],2).isin(np.round(df4.latitude, 2)) & np.round(df3['Longitude'],2).isin(np.round(df4.longitude, 2)))

0       False
1        True
2       False
4        True
7        True
        ...  
1193     True
1194     True
1195     True
1196     True
1197     True
Length: 827, dtype: bool

In [123]:
df5=df4.loc[np.logical_not(np.round(df4.latitude, 2).isin(np.round(df3['Latitude'],2)) & np.round(df4.longitude, 2).isin(np.round(df3['Longitude'],2)))]

In [135]:
for k,i in enumerate(df5.index):
    if k%100==1:print(k/len(df5))
    # try: 
    make_aoi_shape_from_latlons(df5.latitude[i],df5.longitude[i]+0.1, df5['uid'][i], 'E:/Users/sentinel_industry/polys/steel/no_ind/')
    # except ValueError: continue

0.0008968609865470852
0.09058295964125561
0.18026905829596412
0.2699551569506726
0.3596412556053812
0.4493273542600897
0.5390134529147982
0.6286995515695067
0.7183856502242153
0.8080717488789237
0.8977578475336323
0.9874439461883409
